In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def evaluate_rules(data, tuned_rules, output):
    predictions_all = []
    for i, rule in tuned_rules.iterrows():
        pred_rule = []
        tptot = 0
        tntot = 0
        fntot = 0
        fptot = 0
        tuned_antecedent = rule['Rule'].strip()
        print(f"rule: {tuned_antecedent}")
        for _, row in data.iterrows():
            y_row = row[output]
            tp = 0
            tn = 0
            fn = 0
            fp = 0
            # check if the point row satifies rule 
            if evaluate_rule_conditions(row, tuned_antecedent):
                # rule is satisfied
                satisfied = True
                pred_rule.append(0)
            else:
                satisfied = False
                pred_rule.append(1)
    
            
            if satisfied and y_row == 0:
                tp=1
            elif satisfied and y_row == 1:
                fp=1
            elif not satisfied and y_row == 1:
                tn=1
            
            elif not satisfied and y_row == 0:
                fn=1

            predictions_all.append(pred_rule)
            tptot+=tp
            tntot+=tn
            fptot+=fp
            fntot+=fn

        if (tptot+fptot) == 0:
            print("no points satisfy the rule\n")
            continue
        else:
            
            print(f"tp = {tptot}, tn = {tntot}, fp = {fptot}, fn = {fntot}")
            prec = tptot / (tptot+fptot)
            cov = tptot/(tptot+fntot)
            acc = (tptot+tntot)/(tptot+tntot+fptot+fntot)
            f1score = (2*tptot)/(2*tptot+fptot+fntot)
            error = fptot/(fptot+tntot)
            print(f"precision: {prec}, covering: {cov}, error: {error}, accuracy: {acc}, f1: {f1score}\n")              

    return predictions_all


def evaluate_rule_conditions(row, condition_part):

    # Checks if any of the conditions in the rule are satisfied
    if all(check_condition(row, part) for part in condition_part.split(" and ")):
        return True  # Return True if all conditions in the rule are satisfied
    
    return False  # Return False if any of the conditions in the rule is not satisfied


def check_condition(row, condition_part):
    # Check if a single condition part is satisfied
    
    #parts = [part.strip('()') for part in condition_part.split()]
    parts = re.split(r'\s*(==|<=|>=|<|>|!=)\s*', condition_part.strip('()'))
    #print("parts: ", parts)
    if len(parts) == 3:
        column, op, value = parts
        return eval(f"{row[column]} {op} {value}")
    # handle the case of a 2-thresholds conditions of the kind: a < Column <= b
    elif len(parts) == 5:
        val1,op1,column,op2,val2 = parts
        # Use the original condition from the rule
        return eval(f"{val1} {op1} {row[column]} {op2} {val2}")        
    else:
        # Handle cases where the condition is not in the expected format
        pass


In [3]:
# Loading rules actual rules
rules = pd.read_csv('/Users/saranarteni/OneDrive - CNR/REXASI-PRO/T3.1/navground_code/simulation2/confiderai/supsinavground_rules.csv', header=None, names=["Rule", "Covering", "Error"])

# adjust columns values
for i in range(len(rules)+1):   
    rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("AND","and"))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("{",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("}",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("output in ","output = "))
rules["Covering"] = rules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
rules["Error"] = rules["Error"].apply(lambda x: x.replace("ERROR: ",""))
rules['Output'] = rules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace("IF ",""))
rules["Rule"] = rules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))
# load test data
data = pd.read_csv("simulation2/test.csv")

rules_noncoll = rules[rules["Output"]==0]
preds_LLM = evaluate_rules(data, rules_noncoll, "output")

rule: SafetyMargin > 0.071752 and Tau <= 0.798864
tp = 219, tn = 431, fp = 18, fn = 332
precision: 0.9240506329113924, covering: 0.397459165154265, error: 0.0400890868596882, accuracy: 0.65, f1: 0.5558375634517766

rule: SafetyMargin > 0.019271 and 0.096355 < Tau <= 0.359004
tp = 216, tn = 429, fp = 20, fn = 335
precision: 0.9152542372881356, covering: 0.39201451905626133, error: 0.044543429844097995, accuracy: 0.645, f1: 0.5489199491740788

rule: SafetyMargin > 0.070627 and Eta > 0.372908
tp = 192, tn = 441, fp = 8, fn = 359
precision: 0.96, covering: 0.3484573502722323, error: 0.017817371937639197, accuracy: 0.633, f1: 0.511318242343542

rule: 0.041272 < SafetyMargin <= 0.071004 and Tau <= 0.619041
tp = 142, tn = 434, fp = 15, fn = 409
precision: 0.9044585987261147, covering: 0.2577132486388385, error: 0.0334075723830735, accuracy: 0.576, f1: 0.4011299435028249

rule: SafetyMargin > 0.010931 and Eta > 0.748042 and Tau <= 0.682408
tp = 145, tn = 430, fp = 19, fn = 406
precision: 0.884

# Skope Rules performance

In [3]:
#mix both rules, model for collision and model for no collision.
skrules_col = pd.read_csv('simulationVictorTopological/skope/skope_rules_deadlocks.csv', header=None, names=["Rule","Covering","Error"])
skrules_nocol = pd.read_csv('simulationVictorTopological/skope/skope_rules_nondeadlocks.csv', names=["Rule","Covering","Error"])

skrules = pd.concat([skrules_col, skrules_nocol], ignore_index=True)
skrules

,Rule,Covering,Error
0,RULE 1: IF meanEntropy > 2.768405795097351 and...,COVERING: 0.6241935483870967,ERROR: 0.005141388174807249
1,RULE 2: IF medianEntropy > 2.7718499898910522 ...,COVERING: 0.596364683686725,ERROR: 0.001265203024091055
2,RULE 3: IF medianEntropy <= 2.7671250104904175...,COVERING: 0.15255157437567862,ERROR: 0.1380368098159509
3,RULE 4: IF meanEntropy <= 2.7643481492996216 a...,COVERING: 0.14030335861321777,ERROR: 0.11604095563139927
4,RULE 5: IF medianEntropy <= 2.7718499898910522...,COVERING: 0.12703407235358985,ERROR: 0.13006669596957587
5,RULE 6: IF meanEntropy <= 2.769745707511902 an...,COVERING: 0.10278372591006424,ERROR: 0.09859154929577463
6,RULE 7: IF medianEntropy <= 2.7496750354766846...,COVERING: 0.0722041259500543,ERROR: 0.18902439024390238
7,RULE 8: IF meanEntropy > 2.763972520828247 and...,COVERING: 0.07046799354491662,ERROR: 0.096551724137931
8,RULE 9: IF meanEntropy > 2.7211110591888428 an...,COVERING: 0.06952743074416078,ERROR: 0.21472392638036808
9,RULE 10: IF medianEntropy <= 2.761875033378601...,COVERING: 0.06286942504030091,ERROR: 0.48908296943231444


In [4]:
skrules["Rule"][0]

'RULE 1: IF meanEntropy > 2.768405795097351 and medianEntropy > 2.760925054550171 THEN output = 1'

In [5]:
# adjust columns values
for i in range(len(skrules)+1):   
    skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("RULE {}: ".format(i),""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("AND","and"))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("{",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("}",""))
#skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("output in ","output = "))
skrules["Covering"] = skrules["Covering"].apply(lambda x: x.replace("COVERING: ",""))
skrules["Error"] = skrules["Error"].apply(lambda x: x.replace("ERROR: ",""))
skrules['Output'] = skrules['Rule'].str.extract(r'output = (\d+)', expand=False).astype(int)
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace("IF ",""))
skrules["Rule"] = skrules["Rule"].apply(lambda x: x.replace(x[x.find("THEN"):],""))

In [6]:
# for nondead
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_nondead = skrules[skrules["Output"]==0]
preds_skope = evaluate_rules(data, skrules_nondead, "output")

rule: medianEntropy <= 2.7130749225616455
tp = 219, tn = 2, fp = 0, fn = 217
precision: 1.0, covering: 0.5022935779816514, error: 0.0, accuracy: 0.5045662100456622, f1: 0.6687022900763359

rule: meanEntropy <= 2.7211110591888428 and medianEntropy <= 2.7130749225616455
tp = 219, tn = 2, fp = 0, fn = 217
precision: 1.0, covering: 0.5022935779816514, error: 0.0, accuracy: 0.5045662100456622, f1: 0.6687022900763359

rule: meanEntropy > 2.7211110591888428 and medianEntropy <= 2.7497750520706177 and stdsEntropy <= 0.026721810922026634
tp = 117, tn = 2, fp = 0, fn = 319
precision: 1.0, covering: 0.268348623853211, error: 0.0, accuracy: 0.271689497716895, f1: 0.4231464737793852

rule: medianEntropy <= 2.7496750354766846 and medianEntropy > 2.723675012588501 and stdsEntropy <= 0.02671420481055975
tp = 109, tn = 2, fp = 0, fn = 327
precision: 1.0, covering: 0.25, error: 0.0, accuracy: 0.2534246575342466, f1: 0.4

rule: meanEntropy > 2.7087241411209106 and medianEntropy <= 2.7354750633239746
tp =

In [7]:
# for deadlock
data = pd.read_csv("simulationVictorTopological/test.csv")
skrules_dead = skrules[skrules["Output"]==1]
preds_skope = evaluate_rules(data, skrules_nondead, "output")

rule: meanEntropy > 2.768405795097351 and medianEntropy > 2.760925054550171
tp = 2, tn = 2, fp = 0, fn = 434
precision: 1.0, covering: 0.0045871559633027525, error: 0.0, accuracy: 0.0091324200913242, f1: 0.0091324200913242

rule: medianEntropy > 2.7718499898910522
no points satisfy the rule

rule: medianEntropy <= 2.7671250104904175 and medianEntropy > 2.7496750354766846 and iqrsEntropy > 0.030925000086426735
tp = 12, tn = 1, fp = 1, fn = 424
precision: 0.9230769230769231, covering: 0.027522935779816515, error: 0.5, accuracy: 0.02968036529680365, f1: 0.053452115812917596

rule: meanEntropy <= 2.7643481492996216 and medianEntropy > 2.7496750354766846 and iqrsEntropy > 0.03151250071823597
tp = 10, tn = 1, fp = 1, fn = 426
precision: 0.9090909090909091, covering: 0.022935779816513763, error: 0.5, accuracy: 0.02511415525114155, f1: 0.0447427293064877

rule: medianEntropy <= 2.7718499898910522 and medianEntropy > 2.760925054550171
tp = 13, tn = 1, fp = 1, fn = 423
precision: 0.9285714285714